### aim: seasonal decadal averages, 1950-2020 for old and new era. 

sanity check: if something was deeply inconsistent, hopefully we would see it. 

### date: 2022-05-09

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE')
#list of models
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS')
import lom
import utils as ut

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 12})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 12}

plt.rc('font', **font)

In [4]:
nera = f'/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/'
oera = f'/gpfs/data/greenocean/software/products/ERA5Forcing/daily/'


In [5]:
nera

'/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/'

In [6]:
# def make_yearlist(yrst, yrend, baseDir,dtype = 'bulk_10'):
#     yrs = np.arange(yrst,yrend+1,1)
#     ylist = []
    
#     for i in range(0,len(yrs)):
#         #era5_bulk_10_y2019.nc
#         ty = f'{baseDir}/{dtype}_y{yrs[i]}.nc'
#         print(ty)
#         t2 = glob.glob(ty)
#         ylist.append(t2[0])
#     return ylist

# varpat = 'era5_bulk_8'
# tvar = 'air'

# baseDir = nera
# #nera_bulk_8 = get_1950_2020_mean(baseDir,varpat,tvar)
# tylist = make_yearlist(1950,1950+9,baseDir,varpat)

In [7]:
def make_yearlist(yrst, yrend, baseDir,dtype = 'bulk_10'):
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        #era5_bulk_10_y2019.nc
        ty = f'{baseDir}/{dtype}_y{yrs[i]}.nc'
        t2 = glob.glob(ty)
        ylist.append(t2[0])
    return ylist

def get_decadal_mean(ylist, tvar):
    
    decadal_av_stor = np.zeros([10,4,149,182])
    for i in range(0,10):
        tdat = xr.open_dataset(ylist[i], decode_times=False)
        tz = tdat[tvar][0:59,:,:].values
        tz2 = tdat[tvar][334:-1,:,:].values
        tz_djf = np.concatenate((tz,tz2), axis =0)
        tz_mam = tdat[tvar][59:151,:,:].values
        tz_jja = tdat[tvar][151:243,:,:].values
        tz_son = tdat[tvar][243:334,:,:].values
        
        decadal_av_stor[i,1,:,:] = np.nanmean(tz_mam, axis = 0)
        decadal_av_stor[i,2,:,:] = np.nanmean(tz_jja, axis = 0)
        decadal_av_stor[i,3,:,:] = np.nanmean(tz_son, axis = 0)
        decadal_av_stor[i,0,:,:] = np.nanmean(tz_djf, axis = 0)
        
    decadal_avg = np.nanmean(decadal_av_stor, axis = 0)    
    return decadal_avg

def get_1950_2020_mean(baseDir,varpat,tvar):
    
    print(f'{baseDir}, {varpat}, {tvar}')
    decadal_stor = np.zeros([7,4,149,182])
    for i in range(0,7):
        yrst = 1950 + 10*i
        print(yrst)
        tylist = make_yearlist(yrst,yrst+9,baseDir,varpat)
        decadal_avg = get_decadal_mean(tylist, tvar)
        decadal_stor[i,:,:,:] = decadal_avg
        
    return decadal_stor
    

def make_netcdf(eraV, var, tunits, tarray, varnam): 
    savenam = f'./extracted/{eraV}_{var}_1950-2020.nc'
    print(f'making {savenam}')
    data_vars = {varnam:(['decade', 'season', 'y', 'x'], tarray,
    {'units': tunits,
    }),
    }
    # define coordinates
    coords = {'decade': (['decade'], np.arange(1950,2020,10)),
            'season': (['season'], np.arange(0,4,1)),
    'y': (['y'], np.arange(0,149,1)),
    'x': (['x'], np.arange(0,182,1)),
    }
    # define global attributes
    attrs = {'made in':'SOZONE/GRO2_FORCING_EXPERIMENT/ERA5_comparisons-extract.ipynb',
    'desc': 'season 0 is djf, season 1 is mam'
    }
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    #print(ds)
    ds.to_netcdf(savenam)
    return ds 

### bulk 9 

In [8]:
### extract 
varpat = 'era5_bulk_9'
tvar = 'shum'

baseDir = nera
nera_bulk_9 = get_1950_2020_mean(baseDir,varpat,tvar)
baseDir = oera
oera_bulk_9 = get_1950_2020_mean(baseDir,varpat,tvar)

### 
var = 'bulk_9_shum'
tunits = 'kg/kg'
varnam = 'shum'

eraV = 'ERA5_v202303'
tarray = nera_bulk_9
ds = make_netcdf(eraV, var, tunits, tarray, varnam)

eraV = 'ERA5_v2022'
tarray = oera_bulk_9
ds = make_netcdf(eraV, var, tunits, tarray, varnam)

##############pres 
### extract 
varpat = 'era5_bulk_9'
tvar = 'pres'

baseDir = nera
nera_bulk_9 = get_1950_2020_mean(baseDir,varpat,tvar)
baseDir = oera
oera_bulk_9 = get_1950_2020_mean(baseDir,varpat,tvar)

### 
var = 'bulk_9_pres'
tunits = 'kg/m^2/s'
varnam = 'pres'

eraV = 'ERA5_v202303'
tarray = nera_bulk_9
ds = make_netcdf(eraV, var, tunits, tarray, varnam)

eraV = 'ERA5_v2022'
tarray = oera_bulk_9
ds = make_netcdf(eraV, var, tunits, tarray, varnam)


/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/, era5_bulk_9, shum
1950
1960
1970
1980
1990
2000
2010
/gpfs/data/greenocean/software/products/ERA5Forcing/daily/, era5_bulk_9, shum
1950
1960
1970
1980
1990
2000
2010
making ./extracted/ERA5_v202303_bulk_9_shum_1950-2020.nc
making ./extracted/ERA5_v2022_bulk_9_shum_1950-2020.nc
/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/, era5_bulk_9, pres
1950
1960
1970
1980
1990
2000
2010
/gpfs/data/greenocean/software/products/ERA5Forcing/daily/, era5_bulk_9, pres
1950
1960
1970
1980
1990
2000
2010
making ./extracted/ERA5_v202303_bulk_9_pres_1950-2020.nc
making ./extracted/ERA5_v2022_bulk_9_pres_1950-2020.nc


## bulk 8

In [121]:
### extract 
varpat = 'era5_bulk_8'
tvar = 'air'

baseDir = nera
nera_bulk_8 = get_1950_2020_mean(baseDir,varpat,tvar)
baseDir = oera
oera_bulk_8 = get_1950_2020_mean(baseDir,varpat,tvar)

### 
var = 'bulk_8'
tunits = 'degK'
varnam = 'air'

eraV = 'ERA5_v202303'
tarray = nera_bulk_8
ds = make_netcdf(eraV, var, tunits, tarray, varnam)

eraV = 'ERA5_v2022'
tarray = oera_bulk_8
ds = make_netcdf(eraV, var, tunits, tarray, varnam)


/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/, era5_bulk_8, air
1950
1960
1970
1980
1990
2000
2010
/gpfs/data/greenocean/software/products/ERA5Forcing/daily/, era5_bulk_8, air
1950
1960
1970
1980
1990
2000
2010
making ./extracted/ERA5_v202303_bulk_8_1950-2020.nc
making ./extracted/ERA5_v2022_bulk_8_1950-2020.nc


# bulk 11-14. check name and units here

In [129]:
tvar = 'taux_1d'
nera_dat = xr.open_dataset(f'/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/{tvar}_y1950.nc', decode_times=False)
nera_dat.uflx
#nera_dat.prsn
# nera_dat.rlds
#nera_dat.air

<xarray.DataArray 'uflx' (time_counter: 365, y: 149, x: 182)>
[9898070 values with dtype=float32]
Coordinates:
  * time_counter  (time_counter) float32 0.0 86400.0 ... 31363200.0 31449600.0
Dimensions without coordinates: y, x
Attributes:
    units:               m/s
    short_name:          uflx
    axis:                TYX
    interval_operation:  86400.0
    interval_write:      86400.0
    associate:           time_counter nav_lat nav_lon

## bulk 11

In [123]:
### extract 
varpat = 'era5_bulk_11'
var_short = 'bulk_11'
tvar = 'rsds'
tunits = 'W/m^2'
varnam = tvar

baseDir = nera
nera_bulk_11 = get_1950_2020_mean(baseDir,varpat,tvar)
baseDir = oera
oera_bulk_11 = get_1950_2020_mean(baseDir,varpat,tvar)

### 

eraV = 'ERA5_v202303'
tarray = nera_bulk_11
ds = make_netcdf(eraV, var_short, tunits, tarray, varnam)

eraV = 'ERA5_v2022'
tarray = oera_bulk_11
ds = make_netcdf(eraV, var_short, tunits, tarray, varnam)


/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/, era5_bulk_11, rsds
1950
1960
1970
1980
1990
2000
2010
/gpfs/data/greenocean/software/products/ERA5Forcing/daily/, era5_bulk_11, rsds
1950
1960
1970
1980
1990
2000
2010
making ./extracted/ERA5_v202303_bulk_11_1950-2020.nc
making ./extracted/ERA5_v2022_bulk_11_1950-2020.nc


In [124]:
### extract 
varpat = 'era5_bulk_12'
var_short = 'bulk_12'
tvar = 'rlds'
tunits = 'W/m^2'
varnam = tvar

baseDir = nera
nera_bulk_12 = get_1950_2020_mean(baseDir,varpat,tvar)
baseDir = oera
oera_bulk_12 = get_1950_2020_mean(baseDir,varpat,tvar)

### 

eraV = 'ERA5_v202303'
tarray = nera_bulk_12
ds = make_netcdf(eraV, var_short, tunits, tarray, varnam)

eraV = 'ERA5_v2022'
tarray = oera_bulk_12
ds = make_netcdf(eraV, var_short, tunits, tarray, varnam)


/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/, era5_bulk_12, rlds
1950
1960
1970
1980
1990
2000
2010
/gpfs/data/greenocean/software/products/ERA5Forcing/daily/, era5_bulk_12, rlds
1950
1960
1970
1980
1990
2000
2010
making ./extracted/ERA5_v202303_bulk_12_1950-2020.nc
making ./extracted/ERA5_v2022_bulk_12_1950-2020.nc


In [125]:
### extract 
varpat = 'era5_bulk_13'
var_short = 'bulk_13'
tvar = 'prra'
tunits = 'kg/m^2/s'
varnam = tvar

baseDir = nera
nera_bulk_13 = get_1950_2020_mean(baseDir,varpat,tvar)
baseDir = oera
oera_bulk_13 = get_1950_2020_mean(baseDir,varpat,tvar)

### 

eraV = 'ERA5_v202303'
tarray = nera_bulk_13
ds = make_netcdf(eraV, var_short, tunits, tarray, varnam)

eraV = 'ERA5_v2022'
tarray = oera_bulk_13
ds = make_netcdf(eraV, var_short, tunits, tarray, varnam)



/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/, era5_bulk_13, prra
1950
1960
1970
1980
1990
2000
2010
/gpfs/data/greenocean/software/products/ERA5Forcing/daily/, era5_bulk_13, prra
1950
1960
1970
1980
1990
2000
2010
making ./extracted/ERA5_v202303_bulk_13_1950-2020.nc
making ./extracted/ERA5_v2022_bulk_13_1950-2020.nc


In [126]:
### extract 
varpat = 'era5_bulk_14'
var_short = 'bulk_14'
tvar = 'prsn'
tunits = 'kg/m^2/s'
varnam = tvar

baseDir = nera
nera_bulk_14 = get_1950_2020_mean(baseDir,varpat,tvar)
baseDir = oera
oera_bulk_14 = get_1950_2020_mean(baseDir,varpat,tvar)

### 

eraV = 'ERA5_v202303'
tarray = nera_bulk_14
ds = make_netcdf(eraV, var_short, tunits, tarray, varnam)

eraV = 'ERA5_v2022'
tarray = oera_bulk_14
ds = make_netcdf(eraV, var_short, tunits, tarray, varnam)


/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/, era5_bulk_14, prsn
1950
1960
1970
1980
1990
2000
2010
/gpfs/data/greenocean/software/products/ERA5Forcing/daily/, era5_bulk_14, prsn
1950
1960
1970
1980
1990
2000
2010
making ./extracted/ERA5_v202303_bulk_14_1950-2020.nc
making ./extracted/ERA5_v2022_bulk_14_1950-2020.nc


In [130]:
### extract 
varpat = 'taux_1d'
var_short = 'taux'
tvar = 'uflx'
tunits = 'm/s'
varnam = tvar

baseDir = nera
nera_bulk_14 = get_1950_2020_mean(baseDir,varpat,tvar)
baseDir = oera
oera_bulk_14 = get_1950_2020_mean(baseDir,varpat,tvar)

### 

eraV = 'ERA5_v202303'
tarray = nera_bulk_14
ds = make_netcdf(eraV, var_short, tunits, tarray, varnam)

eraV = 'ERA5_v2022'
tarray = oera_bulk_14
ds = make_netcdf(eraV, var_short, tunits, tarray, varnam)


/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/, taux_1d, uflx
1950
1960
1970
1980
1990
2000
2010
/gpfs/data/greenocean/software/products/ERA5Forcing/daily/, taux_1d, uflx
1950
1960
1970
1980
1990
2000
2010
making ./extracted/ERA5_v202303_taux_1950-2020.nc
making ./extracted/ERA5_v2022_taux_1950-2020.nc


In [131]:
### extract 
varpat = 'tauy_1d'
var_short = 'tauy'
tvar = 'vflx'
tunits = 'm/s'
varnam = tvar

baseDir = nera
nera_bulk_14 = get_1950_2020_mean(baseDir,varpat,tvar)
baseDir = oera
oera_bulk_14 = get_1950_2020_mean(baseDir,varpat,tvar)

### 

eraV = 'ERA5_v202303'
tarray = nera_bulk_14
ds = make_netcdf(eraV, var_short, tunits, tarray, varnam)

eraV = 'ERA5_v2022'
tarray = oera_bulk_14
ds = make_netcdf(eraV, var_short, tunits, tarray, varnam)


/gpfs/data/greenocean/software/products/ERA5_v202303_TJ/, tauy_1d, vflx
1950
1960
1970
1980
1990
2000
2010
/gpfs/data/greenocean/software/products/ERA5Forcing/daily/, tauy_1d, vflx
1950
1960
1970
1980
1990
2000
2010
making ./extracted/ERA5_v202303_tauy_1950-2020.nc
making ./extracted/ERA5_v2022_tauy_1950-2020.nc
